# Лабораторная работа №7

## Рекуррентные нейронные сети для анализа текста

Набор данных для предсказания оценок для отзывов, собранных с сайта _imdb.com_, который состоит из 50,000 отзывов в виде текстовых файлов.

Отзывы разделены на положительные (25,000) и отрицательные (25,000).

Данные предварительно токенизированы по принципу «мешка слов», индексы слов можно взять из словаря (_imdb.vocab_).

Обучающая выборка включает в себя 12,500 положительных и 12,500 отрицательных отзывов, контрольная выборка также содержит 12,500 положительных и 12,500 отрицательных отзывов.

Данные можно скачать ~~на сайте _Kaggle_~~: ~~https://www.kaggle.com/iarunava/imdb-movie-reviews-dataset~~ https://ai.stanford.edu/~amaas/data/sentiment/

### Задание 1

Загрузите данные. Преобразуйте текстовые файлы во внутренние структуры данных, которые используют индексы вместо слов.

Будем брать первые `MAX_LENGTH` слов, а если в отзыве слов меньше, чем это число, то применять паддинг.

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

In [0]:
DATA_ARCHIVE_NAME = 'imdb-dataset-of-50k-movie-reviews.zip'

LOCAL_DIR_NAME = 'imdb-sentiments'

In [0]:
from zipfile import ZipFile

with ZipFile(os.path.join(BASE_DIR, DATA_ARCHIVE_NAME), 'r') as zip_:
    zip_.extractall(LOCAL_DIR_NAME)

In [0]:
DATA_FILE_PATH = 'imdb-sentiments/IMDB Dataset.csv'

In [0]:
import pandas as pd

all_df = pd.read_csv(DATA_FILE_PATH)

In [7]:
print(all_df.shape)

(50000, 2)


In [8]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
MAX_LENGTH = 40

STRING_DTYPE = '<U12'

PADDING_TOKEN = 'PAD'

LIMIT_OF_TOKENS = 100000

In [0]:
from nltk import word_tokenize
import numpy as np
import string
import re

def tokenize_string(_string):
    return  [tok_.lower() for tok_ in word_tokenize(_string) if not re.fullmatch('[' + string.punctuation + ']+', tok_)]

def pad(A, length):
    arr = np.empty(length, dtype = STRING_DTYPE)
    arr.fill(PADDING_TOKEN)
    arr[:len(A)] = A
    return arr

def tokenize_row(_sentence):
    return pad(tokenize_string(_sentence)[:MAX_LENGTH], MAX_LENGTH)

def encode_row(_label):
    return 1 if _label == 'positive' else 0

def encode_and_tokenize(_dataframe):

    tttt = _dataframe.apply(lambda row: tokenize_row(row['review']), axis = 1)
    llll = _dataframe.apply(lambda row: encode_row(row['sentiment']), axis = 1)

    data_dict_ = { 'label': llll, 'tokens': tttt }

    encoded_and_tokenized_ = pd.DataFrame(data_dict_, columns = ['label', 'tokens'])

    return encoded_and_tokenized_

In [0]:
all_df_tokenized = encode_and_tokenize(all_df)

In [0]:
from collections import Counter

def get_tokens_list(_dataframe):
    
    all_tokens_ = []
    
    for sent_ in _dataframe['tokens'].values:
        all_tokens_.extend(sent_)

    tokens_counter_ = Counter(all_tokens_)
                
    return [t for t, _ in tokens_counter_.most_common(LIMIT_OF_TOKENS)]

In [0]:
tokens_list = get_tokens_list(all_df_tokenized)

In [0]:
word_to_int_dict = {}

word_to_int_dict.update(
    {t : i for i, t in enumerate(tokens_list)})

In [0]:
def intize_row(_tokens):
    return np.array([word_to_int_dict[t]
                if t in word_to_int_dict
                else 0
            for t in _tokens])

def encode_and_tokenize(_dataframe):

    iiii = _dataframe.apply(lambda row: intize_row(row['tokens']), axis = 1)

    data_dict_ = { 'label': _dataframe['label'], 'ints': iiii }

    intized_ = pd.DataFrame(data_dict_, columns = ['label', 'ints'])

    return intized_

In [0]:
all_df_intized = encode_and_tokenize(all_df_tokenized)

### Задание 2

Реализуйте и обучите двунаправленную рекуррентную сеть (_LSTM_ или _GRU_).

Какого качества классификации удалось достичь?

In [17]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.2.0rc3
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: protobuf, opt-einsum, numpy, tensorflow-estimator, wrapt, scipy, absl-py, gast, wheel, six, google-pasta, termcolor, astunparse, h5py, keras-preprocessing, tensorboard, grpcio
Required-by: 


In [0]:
import tensorflow as tf
from tensorflow import keras

Здесь будем использовать такую конфигурацию рекуррентного _LSTM_-слоя, которая позволит использовать очень быструю _cuDNN_ имплементацию.

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

# The requirements to use the cuDNN implementation are:
# 1. `activation` == `tanh`
# 2. `recurrent_activation` == `sigmoid`
# 3. `recurrent_dropout` == 0
# 4. `unroll` is `False`
# 5. `use_bias` is `True`
# 6. `reset_after` is `True`
# 7. Inputs, if use masking, are strictly right-padded.

model = tf.keras.Sequential()

model.add(Bidirectional(LSTM(100, return_sequences = False), merge_mode = 'concat',
          input_shape = (MAX_LENGTH, 1)))
model.add(Dense(1, activation = 'sigmoid'))

In [20]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 200)               81600     
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 81,801
Trainable params: 81,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
X_intized = np.asarray(list(all_df_intized['ints'].values), dtype = float)[..., np.newaxis]

y_intized = np.asarray(list(all_df_intized['label'].values))

In [22]:
model.fit(x = X_intized, y = y_intized, validation_split = 0.15, epochs = 20)

Epoch 1/20
1329/1329 [==============================] - 10s 7ms/step - loss: 0.6928 - accuracy: 0.5268 - val_loss: 0.6874 - val_accuracy: 0.5453
Epoch 2/20
1329/1329 [==============================] - 9s 7ms/step - loss: 0.6880 - accuracy: 0.5409 - val_loss: 0.6847 - val_accuracy: 0.5464
Epoch 3/20
1329/1329 [==============================] - 9s 7ms/step - loss: 0.6849 - accuracy: 0.5535 - val_loss: 0.6823 - val_accuracy: 0.5535
Epoch 4/20
1329/1329 [==============================] - 10s 7ms/step - loss: 0.6818 - accuracy: 0.5632 - val_loss: 0.6797 - val_accuracy: 0.5632
Epoch 5/20
1329/1329 [==============================] - 9s 7ms/step - loss: 0.6790 - accuracy: 0.5677 - val_loss: 0.6842 - val_accuracy: 0.5569
Epoch 6/20
1329/1329 [==============================] - 9s 7ms/step - loss: 0.6772 - accuracy: 0.5699 - val_loss: 0.6794 - val_accuracy: 0.5651
Epoch 7/20
1329/1329 [==============================] - 9s 7ms/step - loss: 0.6749 - accuracy: 0.5750 - val_loss: 0.6775 - val_accurac

Как и ожидалось, использование эмбеддингов показало лучший результат, чем кодирование просто целыми числами.

### Задание 3

Используйте индексы слов и их различное внутреннее представление (_word2vec_, _glove_). Как влияет данное преобразование на качество классификации?

Используем 300-мерные вектора _FastTest_ &mdash; лучшую на сегодняшний день имплементацию word2vec: https://fasttext.cc/docs/en/english-vectors.html. Файл пришлось доработать &mdash; 9-я строка не читалась.

In [0]:
# VECTORS_ARCHIVE_NAME = 'wiki-news-300d-1M-fixed.zip'

# VECTORS_FILE_NAME = 'wiki-news-300d-1M-fixed.vec'

# VECTORS_LOCAL_DIR_NAME = 'vectors'

In [0]:
# with ZipFile(os.path.join(BASE_DIR, VECTORS_ARCHIVE_NAME), 'r') as zip_:
#     zip_.extractall(VECTORS_LOCAL_DIR_NAME)

Создадим уменьшенный словарь, содержащий только встреченные токены, чтобы уменьшить нагрузку на _Google Drive_:

In [0]:
# def build_vectors_dict(_actual_tokens, _vectors_file_path, _unknown_token = 'unknown'):

#     vec_data_ = pd.read_csv(_vectors_file_path, sep = ' ', header = None, skiprows = [9])
        
#     actual_vectors_ = [x for x in vec_data_.values if x[0] in _actual_tokens or x[0] == _unknown_token]

#     return actual_vectors_

In [0]:
# actual_vectors = build_vectors_dict(tokens_list, os.path.join(VECTORS_LOCAL_DIR_NAME, VECTORS_FILE_NAME))

In [0]:
# vectors_np = np.array(actual_vectors)

# vectors_dict = dict(zip(vectors_np[:, 0], vectors_np[:, 1:]))

# vectors_dict_file_name = 'word-vec-dict-{}-items'.format(len(vectors_dict))

# vectors_dict_file_path = os.path.join(BASE_DIR, vectors_dict_file_name)

# np.savez_compressed(vectors_dict_file_path, vectors_dict, allow_pickle = True)

In [0]:
vectors_dict_file_path = './drive/My Drive/Colab Files/mo-2/word-vec-dict-56485-items.npz'

In [0]:
vectors_dict_data = np.load(vectors_dict_file_path, allow_pickle = True)

vectors_dict = vectors_dict_data['arr_0'][()]

In [0]:
VECTORS_LENGTH = 300

In [0]:
def tokens_to_vectors(_word_to_vec_dict, _tokens, _unknown_token):
    return [_word_to_vec_dict[t]
                if t in _word_to_vec_dict
                else _word_to_vec_dict[_unknown_token]
            for t in _tokens]

def row_to_vectors(_tokens):
    return np.array(tokens_to_vectors(vectors_dict, _tokens, 'unknown'))

def vectorize(_dataframe):

    vvvv = _dataframe.apply(lambda row: row_to_vectors(row['tokens']), axis = 1)

    data_dict_ = { 'label': _dataframe['label'], 'vectors': vvvv }

    vectorized_ = pd.DataFrame(data_dict_, columns = ['label', 'vectors'])

    return vectorized_

In [0]:
all_df_vectorized = vectorize(all_df_tokenized)

In [0]:
X_vectorized = np.asarray(list(all_df_vectorized['vectors'].values), dtype = float)

y_vectorized = np.asarray(list(all_df_intized['label'].values))

In [0]:
model = tf.keras.Sequential()

model.add(Bidirectional(LSTM(100, return_sequences = False), merge_mode = 'concat',
          input_shape = (MAX_LENGTH, VECTORS_LENGTH)))
model.add(Dense(1, activation = 'sigmoid'))

In [38]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 321,001
Trainable params: 321,001
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(x = X_vectorized, y = y_vectorized, validation_split = 0.15, epochs = 20)

Epoch 1/20
1329/1329 [==============================] - 12s 9ms/step - loss: 0.5417 - accuracy: 0.7183 - val_loss: 0.4969 - val_accuracy: 0.7532
Epoch 2/20
1329/1329 [==============================] - 11s 8ms/step - loss: 0.4844 - accuracy: 0.7581 - val_loss: 0.4829 - val_accuracy: 0.7536
Epoch 3/20
1329/1329 [==============================] - 11s 9ms/step - loss: 0.4558 - accuracy: 0.7754 - val_loss: 0.4961 - val_accuracy: 0.7408
Epoch 4/20
1329/1329 [==============================] - 11s 9ms/step - loss: 0.4314 - accuracy: 0.7914 - val_loss: 0.4763 - val_accuracy: 0.7564
Epoch 5/20
1329/1329 [==============================] - 11s 8ms/step - loss: 0.4060 - accuracy: 0.8066 - val_loss: 0.4622 - val_accuracy: 0.7737
Epoch 6/20
1329/1329 [==============================] - 11s 8ms/step - loss: 0.3754 - accuracy: 0.8238 - val_loss: 0.4739 - val_accuracy: 0.7683
Epoch 7/20
1329/1329 [==============================] - 11s 8ms/step - loss: 0.3387 - accuracy: 0.8454 - val_loss: 0.5071 - val_ac

### Задание 4

Поэкспериментируйте со структурой сети (добавьте больше рекуррентных, полносвязных или сверточных слоев). Как это повлияло на качество классификации?

### Задание 5

Используйте предобученную рекуррентную нейронную сеть (например, _DeepMoji_ или что-то подобное).

Какой максимальный результат удалось получить на контрольной выборке?